In [141]:
!git clone https://github.com/Pyroxene889/Store-Sales

fatal: destination path 'Store-Sales' already exists and is not an empty directory.


In [142]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [143]:
import opendatasets as od
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

# Data Cleaning

## Data Collecting

In [144]:
od.download(
	"https://www.kaggle.com/competitions/store-sales-time-series-forecasting/data?select=holidays_events.csv", force = True)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: ikramlaaouane
Your Kaggle Key: ··········


100%|██████████| 21.4M/21.4M [00:00<00:00, 154MB/s]


Extracting archive ./store-sales-time-series-forecasting/store-sales-time-series-forecasting.zip to ./store-sales-time-series-forecasting


In [145]:
df_train = pd.read_csv('/content/store-sales-time-series-forecasting/train.csv')

In [146]:
df_stores = pd.read_csv('/content/store-sales-time-series-forecasting/stores.csv')

In [147]:
df = df_train.set_index('store_nbr').join(df_stores.set_index('store_nbr'))
df = df.drop(['id'], axis = 1)
df.head()

,date,family,sales,onpromotion,city,state,type,cluster
store_nbr,,,,,,,,
1,2013-01-01,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13
1,2013-01-01,BABY CARE,0.0,0,Quito,Pichincha,D,13
1,2013-01-01,BEAUTY,0.0,0,Quito,Pichincha,D,13
1,2013-01-01,BEVERAGES,0.0,0,Quito,Pichincha,D,13
1,2013-01-01,BOOKS,0.0,0,Quito,Pichincha,D,13


In [148]:
df.isnull().sum()

date           0
family         0
sales          0
onpromotion    0
city           0
state          0
type           0
cluster        0
dtype: int64

In [149]:
df_holidays = pd.read_csv('/content/store-sales-time-series-forecasting/holidays_events.csv')
df_holidays["type"].value_counts().index

Index(['Holiday', 'Event', 'Additional', 'Transfer', 'Bridge', 'Work Day'], dtype='object')

In [150]:
df_holidays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [151]:
df = df.set_index('date').join(df_holidays.set_index('date'), lsuffix='_store', rsuffix='_holiday')
df.head()

,family,sales,onpromotion,city,state,type_store,cluster,type_holiday,locale,locale_name,description,transferred
date,,,,,,,,,,,,
2013-01-01,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
2013-01-01,BABY CARE,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
2013-01-01,BEAUTY,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
2013-01-01,BEVERAGES,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
2013-01-01,BOOKS,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False


In [152]:
df["type_holiday"].value_counts()

Holiday       304722
Event          99792
Additional     67716
Transfer       16038
Work Day        8910
Bridge          5346
Name: type_holiday, dtype: int64

In [153]:
# Transformer les jours fériers qui ont été transférré en un autre jour en jour normal "work Day"
df.loc[df['transferred'] == True, 'type_holiday'] = 'Work Day'
df.head()

,family,sales,onpromotion,city,state,type_store,cluster,type_holiday,locale,locale_name,description,transferred
date,,,,,,,,,,,,
2013-01-01,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
2013-01-01,BABY CARE,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
2013-01-01,BEAUTY,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
2013-01-01,BEVERAGES,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
2013-01-01,BOOKS,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False


In [154]:
df["type_holiday"].value_counts()

Holiday       288684
Event          99792
Additional     67716
Work Day       24948
Transfer       16038
Bridge          5346
Name: type_holiday, dtype: int64

In [155]:
#Supprimer la colonne "locale_name", puisqu'on n'a pas besoin
df = df.drop(['locale_name'], axis = 1)
df = df.drop(['description'], axis = 1)
df = df.drop(['transferred'], axis = 1)
df.head()

,family,sales,onpromotion,city,state,type_store,cluster,type_holiday,locale
date,,,,,,,,,
2013-01-01,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,Holiday,National
2013-01-01,BABY CARE,0.0,0,Quito,Pichincha,D,13,Holiday,National
2013-01-01,BEAUTY,0.0,0,Quito,Pichincha,D,13,Holiday,National
2013-01-01,BEVERAGES,0.0,0,Quito,Pichincha,D,13,Holiday,National
2013-01-01,BOOKS,0.0,0,Quito,Pichincha,D,13,Holiday,National


In [156]:
df.isnull().sum()

family                0
sales                 0
onpromotion           0
city                  0
state                 0
type_store            0
cluster               0
type_holiday    2551824
locale          2551824
dtype: int64

In [157]:
df["type_holiday"].fillna("work", inplace = True)
df.isnull().sum()

family                0
sales                 0
onpromotion           0
city                  0
state                 0
type_store            0
cluster               0
type_holiday          0
locale          2551824
dtype: int64

In [158]:
df["locale"].value_counts()

National    261954
Local       208494
Regional     32076
Name: locale, dtype: int64

In [159]:
df["locale"].fillna("National", inplace = True)
df.isnull().sum()

family          0
sales           0
onpromotion     0
city            0
state           0
type_store      0
cluster         0
type_holiday    0
locale          0
dtype: int64

In [164]:
df.head()

,family,sales,onpromotion,city,state,type_store,cluster,type_holiday,locale
date,,,,,,,,,
2013-01-01,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,Holiday,National
2013-01-01,BABY CARE,0.0,0,Quito,Pichincha,D,13,Holiday,National
2013-01-01,BEAUTY,0.0,0,Quito,Pichincha,D,13,Holiday,National
2013-01-01,BEVERAGES,0.0,0,Quito,Pichincha,D,13,Holiday,National
2013-01-01,BOOKS,0.0,0,Quito,Pichincha,D,13,Holiday,National


In [160]:
df_oil = pd.read_csv('/content/store-sales-time-series-forecasting/oil.csv')
df_oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [166]:
df = df.set_index("date").join(df_oil.set_index('date'))
df

KeyError: ignored

In [161]:
def fillna_oil(df):
  df2=df
  df2['oil2']=df2['dcoilwtico'].shift(1)
  df2['oil3']=df2['dcoilwtico'].shift(-1)

  df2.loc[df2['dcoilwtico'].isnull(),'dcoilwtico']=df2['oil2']
  df2.loc[df2['dcoilwtico'].isnull(),'dcoilwtico']=df2['oil3']

  df2.drop(['oil2', 'oil3'], axis=1,inplace=True)
  df2.head()

  df2.head(20)
  return df2

In [ ]:
df2 = fillna_oil(df)